In [1]:
#!pip install surprise

# Sistema Recomendador de Contenido

### Cargar los datos

In [2]:
# Cargar el dataset con la información de las actividades disponibles
import pandas as pd

# Leemos los datos del fichero y lo cargamos en un dataframe
actividades = pd.read_json("https://raw.githubusercontent.com/Dacilbg/Actividad/main/Actividades.json")
actividades.info()

# Mostramos las primeras filas para saber que se ha cargado correctamente
actividades.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   idActividad            84 non-null     int64 
 1   Nombre_Actividad       84 non-null     object
 2   Categoría              84 non-null     object
 3   Fase_del_Alzheimer     84 non-null     object
 4   Característica         84 non-null     object
 5   Número_personas        84 non-null     object
 6   Descripción_actividad  84 non-null     object
 7   Material_necesario     84 non-null     object
dtypes: int64(1), object(7)
memory usage: 5.4+ KB


,idActividad,Nombre_Actividad,Categoría,Fase_del_Alzheimer,Característica,Número_personas,Descripción_actividad,Material_necesario
0,1,Parchís,Juegos,"[0, 1]",Juegos,De 2 a 4 jugadores,Cada jugador elige uno de los cuatro colores d...,"Un tablero de parchís, un dado y cuatro fichas..."
1,2,La Oca,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,"Cada jugador elige una ficha, con ella tendrá ...","Un tablero de la oca, un dado y una ficha por ..."
2,3,Dominó,Juegos,"[0, 1]","Juegos, Deporte",De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.
3,4,Dominó parejas,Juegos,[2],"Juegos, Deporte",De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.
4,5,Bingo,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,En este juego debe haber una persona que obten...,Las 90 bolas del bingo y cartones con los núme...


In [3]:
#Cargamos el dataset con los datos de los pacientes
pacientes = pd.read_json("https://raw.githubusercontent.com/Dacilbg/Actividad/main/pacientes.json")
pacientes.info()

# Mostramos las primeras filas para saber que se ha cargado correctamente
pacientes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_Paciente           41 non-null     int64 
 1   Nombre_Paciente       41 non-null     object
 2   Género                41 non-null     int64 
 3   Edad                  41 non-null     int64 
 4   Fase_del_Alzheimer    41 non-null     int64 
 5   Grado_de_dependencia  41 non-null     int64 
 6   Característica        41 non-null     object
 7   Valoraciones          41 non-null     object
dtypes: int64(5), object(3)
memory usage: 2.7+ KB


,id_Paciente,Nombre_Paciente,Género,Edad,Fase_del_Alzheimer,Grado_de_dependencia,Característica,Valoraciones
0,0,Ejemplo,0,0,0,0,"Cocina, Costura, Deporte, Jardinería, Juegos, ...",[]
1,1,Juan,0,75,0,0,"Jardinería, Cocina, Juegos, Música",[]
2,2,Juana,1,83,1,1,"Costura, Juegos, Lectura, Manualidad",[]
3,3,Pedro,0,68,1,1,"Pintar, Teatro, Música, Lectura",[]
4,4,Manuela,1,79,3,3,"Manualidad, Lectura, Costura, Jardinería",[]


### Prepara los datos

#### - Dataset de las actividades:

A continuación, vamos a preparar los datos para que podamos usar el algoritmo. Lo que necesitamos hacer es cambiar el formato en el que tenemos almacenadas las características, quitar el símbolo `, ` y convertirlo a un espacio en blanco.

Para hacer esta conversión, aplicamos la función `apply` a la columna `Característica`. Luego, por cada característica que leemos usamos la función `replace` que nos permite hacer la sustitución de los caracteres.


In [4]:
# Preparar los datos

# En la columna Característica, cambialos los caracteres ', ' por espacios en blanco
actividades['Característica'] = actividades['Característica'].apply(lambda Característica:
                                                                    Característica.replace(', ',' '))

# Mostramos las primeras filas para comprobar que lo hemos hecho correctamente
actividades.head()

,idActividad,Nombre_Actividad,Categoría,Fase_del_Alzheimer,Característica,Número_personas,Descripción_actividad,Material_necesario
0,1,Parchís,Juegos,"[0, 1]",Juegos,De 2 a 4 jugadores,Cada jugador elige uno de los cuatro colores d...,"Un tablero de parchís, un dado y cuatro fichas..."
1,2,La Oca,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,"Cada jugador elige una ficha, con ella tendrá ...","Un tablero de la oca, un dado y una ficha por ..."
2,3,Dominó,Juegos,"[0, 1]",Juegos Deporte,De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.
3,4,Dominó parejas,Juegos,[2],Juegos Deporte,De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.
4,5,Bingo,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,En este juego debe haber una persona que obten...,Las 90 bolas del bingo y cartones con los núme...


Vamos a extraer las Fases de Alzheimer para cada actividad. De este modo sabremos que actividades están orientadas para cada fase del Alzheimer. Dividimos en tres fases:
- 0: no ha sido diagnosticado con Alzheimer
- 1: Alzheimer leve
- 2: Alzheimer moderado

Para cada fase vamos a crear un dataframe independiente que recoja las actividades destinadas a dicha fase del Alzheimer

In [5]:
actividades_df = actividades.copy()
for index, row in actividades.iterrows():
    for fase in row['Fase_del_Alzheimer']:
        if fase == 0:
            actividades_df.at[index,'Fase_0'] = 1
        if fase == 1:
            actividades_df.at[index,'Fase_1'] = 1
        if fase == 2:
            actividades_df.at[index,'Fase_2'] = 1

actividades_df = actividades_df.fillna(0)
actividades_df.head()

,idActividad,Nombre_Actividad,Categoría,Fase_del_Alzheimer,Característica,Número_personas,Descripción_actividad,Material_necesario,Fase_0,Fase_1,Fase_2
0,1,Parchís,Juegos,"[0, 1]",Juegos,De 2 a 4 jugadores,Cada jugador elige uno de los cuatro colores d...,"Un tablero de parchís, un dado y cuatro fichas...",1.0,1.0,0.0
1,2,La Oca,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,"Cada jugador elige una ficha, con ella tendrá ...","Un tablero de la oca, un dado y una ficha por ...",1.0,1.0,0.0
2,3,Dominó,Juegos,"[0, 1]",Juegos Deporte,De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.,1.0,1.0,0.0
3,4,Dominó parejas,Juegos,[2],Juegos Deporte,De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.,0.0,0.0,1.0
4,5,Bingo,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,En este juego debe haber una persona que obten...,Las 90 bolas del bingo y cartones con los núme...,1.0,1.0,0.0


In [6]:
actividades_df_Fase_0 = actividades_df.drop(actividades_df[actividades_df['Fase_0']== 0].index)
actividades_df_Fase_0.head()

,idActividad,Nombre_Actividad,Categoría,Fase_del_Alzheimer,Característica,Número_personas,Descripción_actividad,Material_necesario,Fase_0,Fase_1,Fase_2
0,1,Parchís,Juegos,"[0, 1]",Juegos,De 2 a 4 jugadores,Cada jugador elige uno de los cuatro colores d...,"Un tablero de parchís, un dado y cuatro fichas...",1.0,1.0,0.0
1,2,La Oca,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,"Cada jugador elige una ficha, con ella tendrá ...","Un tablero de la oca, un dado y una ficha por ...",1.0,1.0,0.0
2,3,Dominó,Juegos,"[0, 1]",Juegos Deporte,De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.,1.0,1.0,0.0
4,5,Bingo,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,En este juego debe haber una persona que obten...,Las 90 bolas del bingo y cartones con los núme...,1.0,1.0,0.0
6,7,La Ronda Robada,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,Se reparten 3 cartas de la baraja a cada jugad...,Baraja de cartas españolas,1.0,1.0,0.0


In [7]:
actividades_df_Fase_1 = actividades_df.drop(actividades_df[actividades_df['Fase_1']== 0].index)
actividades_df_Fase_1.head()

,idActividad,Nombre_Actividad,Categoría,Fase_del_Alzheimer,Característica,Número_personas,Descripción_actividad,Material_necesario,Fase_0,Fase_1,Fase_2
0,1,Parchís,Juegos,"[0, 1]",Juegos,De 2 a 4 jugadores,Cada jugador elige uno de los cuatro colores d...,"Un tablero de parchís, un dado y cuatro fichas...",1.0,1.0,0.0
1,2,La Oca,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,"Cada jugador elige una ficha, con ella tendrá ...","Un tablero de la oca, un dado y una ficha por ...",1.0,1.0,0.0
2,3,Dominó,Juegos,"[0, 1]",Juegos Deporte,De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.,1.0,1.0,0.0
4,5,Bingo,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,En este juego debe haber una persona que obten...,Las 90 bolas del bingo y cartones con los núme...,1.0,1.0,0.0
6,7,La Ronda Robada,Juegos,"[0, 1]",Juegos,Mínimo 2 jugadores,Se reparten 3 cartas de la baraja a cada jugad...,Baraja de cartas españolas,1.0,1.0,0.0


In [8]:
actividades_df_Fase_2 = actividades_df.drop(actividades_df[actividades_df['Fase_2']== 0].index)
actividades_df_Fase_2.head()

,idActividad,Nombre_Actividad,Categoría,Fase_del_Alzheimer,Característica,Número_personas,Descripción_actividad,Material_necesario,Fase_0,Fase_1,Fase_2
3,4,Dominó parejas,Juegos,[2],Juegos Deporte,De 2 a 4 jugadores,Colocamos las 28 fichas que componen un dominó...,Un dominó.,0.0,0.0,1.0
5,6,Bingo ordenado,Juegos,[2],Juegos,Mínimo 2 jugadores,Pondremos las bolas del Bingo del número 1 al ...,Las 90 bolas del bingo.,0.0,0.0,1.0
7,8,Busca la pareja,Juegos,"[0, 1, 2]",Juegos,Mínimo 2 jugadores,Colocaremos toda la baraja de cartas boca abaj...,Baraja de cartas españolas,1.0,1.0,1.0
8,9,Recopila cartas,Juegos,"[0, 1, 2]",Juegos,Mínimo 1 jugador,Colocaremos todas las cartas de la baraja boca...,Baraja de cartas españolas,1.0,1.0,1.0
9,10,Recopila cartas por números,Juegos,"[0, 1, 2]",Juegos,Mínimo 1 jugador,Colocaremos todas las cartas de la baraja boca...,Baraja de cartas españolas,1.0,1.0,1.0


Otra cosa que vamos a crear es un *mapa inverso*, el cual tendrá como índices los nombres de las actividades y nos devolverá el identificador en el dataframe original (que lo hemos llamado `actividades`).

In [9]:
# Construimos un mapa inverso para obtener los indices a partir de los nombres de las actividades
indices = pd.Series(actividades.index, index=actividades['Nombre_Actividad']).drop_duplicates()

# Mostramos las primeras filas para comprobar que lo hemos hecho correctamente
indices[:5]

Nombre_Actividad
Parchís           0
La Oca            1
Dominó            2
Dominó parejas    3
Bingo             4
dtype: int64

#### - Dataset de los pacientes:

A continuación, vamos a preparar los datos para que podamos usar el algoritmo. Lo que necesitamos hacer es cambiar el formato en el que tenemos almacenadas las características, vamos a convertirlo en una lista de características.

Usaremos la técnica One Hot Encoding para convertir la lista de características en un vector, donde cada columna corresponde a un valor de la característica mencionada. Los valores que tendrán en cada columna serán de 0 ó 1, de tal manera que el valor será 1, en el caso de que dicha característica sea uno de los gustos del paciente, y por el contrario será 0, si el paciente no ha elegido esa característica como de su gusto.

In [10]:
#Cada característica está separada por ', ' para simplificar vamos a convertirlo en lista

pacientes['Característica'] = pacientes.Característica.str.split(', ')
pacientes.head()

,id_Paciente,Nombre_Paciente,Género,Edad,Fase_del_Alzheimer,Grado_de_dependencia,Característica,Valoraciones
0,0,Ejemplo,0,0,0,0,"[Cocina, Costura, Deporte, Jardinería, Juegos,...",[]
1,1,Juan,0,75,0,0,"[Jardinería, Cocina, Juegos, Música]",[]
2,2,Juana,1,83,1,1,"[Costura, Juegos, Lectura, Manualidad]",[]
3,3,Pedro,0,68,1,1,"[Pintar, Teatro, Música, Lectura]",[]
4,4,Manuela,1,79,3,3,"[Manualidad, Lectura, Costura, Jardinería]",[]


In [11]:
#Realizamos una copia del dataset de pacientes para trabajar sobre ella
pacientes_df = pacientes.copy()

#Para cada fila (es decir, cada paciente), iterar la lista de características y
#colocar un 1 en la columna que corresponda
for index, row in pacientes.iterrows():
    for caracteristica in row['Característica']:
        pacientes_df.at[index, caracteristica] = 1

#Completar los valores NaN con 0 para mostrar que a dicho paciente no le gusta la característica de la columna
pacientes_df = pacientes_df.fillna(0)
pacientes_df.head()

,id_Paciente,Nombre_Paciente,Género,Edad,Fase_del_Alzheimer,Grado_de_dependencia,Característica,Valoraciones,Cocina,Costura,Deporte,Jardinería,Juegos,Lectura,Manualidad,Música,Pintar,Teatro
0,0,Ejemplo,0,0,0,0,"[Cocina, Costura, Deporte, Jardinería, Juegos,...",[],1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,Juan,0,75,0,0,"[Jardinería, Cocina, Juegos, Música]",[],1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2,Juana,1,83,1,1,"[Costura, Juegos, Lectura, Manualidad]",[],0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,3,Pedro,0,68,1,1,"[Pintar, Teatro, Música, Lectura]",[],0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,4,Manuela,1,79,3,3,"[Manualidad, Lectura, Costura, Jardinería]",[],0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


### Configuramos el algoritmo de recomendación

Ahora vamos a preparar el algoritmo de recomendación. 

Vamos a utilizar la función `TfidVectorizer` de `sklearn`. Esta función nos devolverá una matriz donde las 
columnas contienen todas las características que existen en el dataset y cada fila representa un paciente. Cada celda de la matriz representa la frecuencia de esa característica en dicho paciente. En este caso tendremos como valor de frecuencia 1 o 0.

Vamos a crear una matriz para cada dataframe que hemos generado anteriormente, dependiendo de la fase del Alzheimer:

In [12]:
# Configurar el sistema recomendador
# Usamos el algoritmo TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Configuramos la función 
tfidf = TfidfVectorizer()

# Obtenemos la matriz de frecuencias pasandole la columna Característica del dataframe actividades
tfidf_matriz_Fase_0 = tfidf.fit_transform(actividades_df_Fase_0.Característica)

# Mostramos las dimensiones de la matriz para comprobar si se ha realizado correctamente
tfidf_matriz_Fase_0.shape

(64, 10)

In [13]:
# Configurar el sistema recomendador
# Usamos el algoritmo TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Configuramos la función 
tfidf = TfidfVectorizer()

# Obtenemos la matriz de frecuencias pasandole la columna Característica del dataframe actividades
tfidf_matriz_Fase_1 = tfidf.fit_transform(actividades_df_Fase_1.Característica)

# Mostramos las dimensiones de la matriz para comprobar si se ha realizado correctamente
tfidf_matriz_Fase_1.shape

(64, 10)

In [14]:
# Configurar el sistema recomendador
# Usamos el algoritmo TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Configuramos la función 
tfidf = TfidfVectorizer()

# Obtenemos la matriz de frecuencias pasandole la columna Característica del dataframe actividades
tfidf_matriz_Fase_2 = tfidf.fit_transform(actividades_df_Fase_2.Característica)

# Mostramos las dimensiones de la matriz para comprobar si se ha realizado correctamente
tfidf_matriz_Fase_2.shape

(55, 10)

#### Elegimos un número aleatorio de paciente entre el listado existente

In [185]:
import random
numero_paciente = random.randint(1,40)
numero_paciente

32

Vamos a ver los valores que tiene el dataset para el paciente seleccionado:

In [186]:
paciente = pacientes_df.iloc[numero_paciente]
paciente

id_Paciente                                                32
Nombre_Paciente                                        Paloma
Género                                                      1
Edad                                                       72
Fase_del_Alzheimer                                          2
Grado_de_dependencia                                        0
Característica          [Lectura, Manualidad, Pintar, Juegos]
Valoraciones                                               []
Cocina                                                    0.0
Costura                                                   0.0
Deporte                                                   0.0
Jardinería                                                0.0
Juegos                                                    1.0
Lectura                                                   1.0
Manualidad                                                1.0
Música                                                    0.0
Pintar  

Vamos a quedarnos con la parte del dataframe de pacientes, que tiene la información de las características que le gustan a cada paciente. Esto nos servirá para definir el perfil de cada uno de nuestros pacietnes.

In [187]:
pacientes_df.shape

(41, 18)

In [188]:
paciente_caracteristicas = pacientes_df.iloc[:,8:18]
paciente_caracteristicas

,Cocina,Costura,Deporte,Jardinería,Juegos,Lectura,Manualidad,Música,Pintar,Teatro
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
6,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
8,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


En este punto, elegimos la fila que corresponde al paciente sobre el que queremos comprobar el recomendador de actividades. Convertiremos los valores de las celdas de su fila en un vector que utilizaremos posteriormente.

In [189]:
paciente_gustos = paciente_caracteristicas.iloc[numero_paciente].to_numpy()
paciente_gustos

array([0., 0., 0., 0., 1., 1., 1., 0., 1., 0.])

Ahora vamos a multiplicar la matriz que habíamos obtenido anteriormente con la frecuencia de las características en las diferentes actividades, por los gustos del paciente que hemos decidido recomendar. 

En el caso de que haya algún paciente con Alzheimer avanzado, se imprimirá un mensaje por pantalla, que informará no hay recomendación para el paciente. Ya que este recomendador se ha creado para pacientes hasta nivel de Alzheimer moderado.

In [190]:
if paciente.Fase_del_Alzheimer == 0:
    recomendacion_paciente = tfidf_matriz_Fase_0.dot(paciente_gustos)
elif paciente.Fase_del_Alzheimer == 1:
    recomendacion_paciente = tfidf_matriz_Fase_1.dot(paciente_gustos)
elif paciente.Fase_del_Alzheimer == 2:
    recomendacion_paciente = tfidf_matriz_Fase_2.dot(paciente_gustos)
elif paciente.Fase_del_Alzheimer == 3:
    print('No hay recomendación para este paciente')
recomendacion_paciente

array([0.54309237, 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.35015432, 0.34480647, 1.33879144,
       1.33879144, 1.        , 0.54309237, 0.54309237, 0.54309237,
       0.54309237, 0.54309237, 0.54309237, 1.        , 1.33879144,
       1.64495905, 0.39434244, 1.        , 0.54309237, 0.54309237,
       1.36071658, 1.36071658, 0.59815615, 1.33879144, 0.89884464,
       0.5795104 , 0.5795104 , 0.5795104 , 1.        , 0.5795104 ,
       0.5795104 , 0.4674937 , 0.4674937 , 1.        , 1.        ,
       0.4674937 , 0.4674937 , 1.36071658, 0.5795104 , 0.5795104 ,
       0.5795104 , 0.5795104 , 0.4674937 , 1.        , 0.5795104 ])

### Usar el sistema recomendador que hemos usado

Una vez que tenemos configurada los valores de las posibles recomendaciones a nuestro paciente, vamos a recomendar las actividades que pueden gustarle.

In [191]:
#Teniendo el array de los valores de los gustos del paciente relacionado con las actividades,
#vamos a convertirlo en una lista para manejarlo mejor
gustos_similares_all = list(enumerate(recomendacion_paciente))
print(gustos_similares_all)

#Ordenamos estos valores de mayor a menor
gustos_similares = sorted(gustos_similares_all, key=lambda x: x[1], reverse=True)

#Nos quedamos con las 10 actividades más similares a los gustos de nuestro paciente
gustos_similares = gustos_similares[0:10]

#Obtenemos los índices de las actividades que vamos a recomendar
indices_actividades = [i[0] for i in gustos_similares]

#Imprimimos por pantalla las actividades que le recomendamos al paciente según los gustos que ha definido
actividades['Nombre_Actividad'].iloc[indices_actividades]

[(0, 0.5430923677878643), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 0.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 0.35015432264617263), (13, 0.34480647023514094), (14, 1.3387914356463255), (15, 1.3387914356463255), (16, 1.0), (17, 0.5430923677878643), (18, 0.5430923677878643), (19, 0.5430923677878643), (20, 0.5430923677878643), (21, 0.5430923677878643), (22, 0.5430923677878643), (23, 1.0), (24, 1.3387914356463255), (25, 1.6449590536073335), (26, 0.39434244162100385), (27, 1.0), (28, 0.5430923677878643), (29, 0.5430923677878643), (30, 1.3607165830865466), (31, 1.3607165830865466), (32, 0.5981561475248088), (33, 1.3387914356463255), (34, 0.8988446437908375), (35, 0.5795103999013562), (36, 0.5795103999013562), (37, 0.5795103999013562), (38, 1.0), (39, 0.5795103999013562), (40, 0.5795103999013562), (41, 0.467493699170923), (42, 0.467493699170923), (43, 1.0), (44, 1.0), (45, 0.467493699170923), (46, 0.467493699170923), (47, 1.3607165830865466), (48, 0.5795103999

25           Puzles de refranes escritos
30    Cada oveja con su pareja o familia
31                Lanzamiento de objetos
47                   Ejercicios cruzados
14                     Parejas de naipes
15        Parejas de naipes, boca arriba
24                    Continua el refrán
33                       Pasar la pelota
1                                 La Oca
2                                 Dominó
Name: Nombre_Actividad, dtype: object

# Sistema Recomendador Colaborativo

En este momento, después de desarrollar la actividad el paciente deberá evaluarla. Las valoraciones se harán teniendo en cuenta lo siguiente: 

- 5 = me encantó la actividad 
- 4 = me gustó la actividad 
- 3 = es útil la actividad 
- 2 = no es útil la actividad 
- 1 = no me gustó la actividad.

Vamos a seleccionar una actividad y después obtener la valoración que el paciente le daría de forma aleatoria:

In [213]:
# Elegimos una actividad
indice_actividad = indices_actividades[7]
indice_actividad

33

In [214]:
# Obtenemos la valoración del paciente sobre la actividad de forma aleatoria 
# Esto lo hacemos para poder evaluar la segunda parte del recomendador
valoracion = random.randint(1,5)
valoracion

5

In [215]:
# Vamos a guardar en la columna Valoraciones, todas las actividades que va realizando el paciente junto con su valoración
paciente.Valoraciones.append([indice_actividad, valoracion])
paciente

id_Paciente                                                            32
Nombre_Paciente                                                    Paloma
Género                                                                  1
Edad                                                                   72
Fase_del_Alzheimer                                                      2
Grado_de_dependencia                                                    0
Característica                      [Lectura, Manualidad, Pintar, Juegos]
Valoraciones            [[1, 4], [25, 5], [30, 5], [31, 4], [47, 3], [...
Cocina                                                                0.0
Costura                                                               0.0
Deporte                                                               0.0
Jardinería                                                            0.0
Juegos                                                                1.0
Lectura                               

### Vamos a generar un nuevo DataFrame que recoja las valoraciones de diferentes pacientes, para poder crear el sistema recomendador colaborativo. 

Para ello necesitamos varias valoraciones, así que ejecutaremos estas celdas anteriores un par de veces para cada paciente. En la práctica tendrían que ser las valoraciones de los pacientes a medida que realizan las actividades, pero esto lo haremos de prueba.

In [57]:
# Ejecutar esta celda una sola vez, para crear el DataFrame que guardará las valoraciones. 
df = pd.DataFrame()

In [216]:
# Creamos un nuevo DataFrame que recoja todas las valoraciones de los diferentes pacientes
# Esta celda debemos ejecutarla cuando ya tengamos guardadas todas las valoraciones de un paciente
if paciente.Valoraciones != []:
    for pacientes in paciente.Valoraciones:
        df = df.append({'user': paciente.id_Paciente,'item': (pacientes[0]+1),'rating': pacientes[1]}, ignore_index=True)
df

,user,item,rating
0,33.0,28.0,4.0
1,33.0,32.0,3.0
2,33.0,26.0,1.0
3,33.0,24.0,1.0
4,33.0,25.0,3.0
5,33.0,35.0,2.0
6,33.0,27.0,5.0
7,31.0,21.0,2.0
8,31.0,37.0,1.0
9,31.0,40.0,2.0


In [217]:
# Convertimos los valores de las columnas 'user' e 'item' en valores enteros
df['user'] = df['user'].astype('int')
df['item'] = df['item'].astype('int')
df

,user,item,rating
0,33,28,4.0
1,33,32,3.0
2,33,26,1.0
3,33,24,1.0
4,33,25,3.0
5,33,35,2.0
6,33,27,5.0
7,31,21,2.0
8,31,37,1.0
9,31,40,2.0


Para que el Sistema Recomendador Colaborativo funcione, tiene que tener como mínimo valoraciones de dos pacientes diferentes. Por lo que, solo crearemos el fichero .csv que vamos a necesitar posteriormente, en el caso de que tengamos valoraciones de más de dos pacietnes.

In [218]:
# Convertimos el DataFrame de las valoraciones en un documento .csv
if len(df['user'].unique()) > 2:
    df.to_csv('rating.csv', index = False)

Para cargar los datos de un fichero en `surprise`, vamos a utilizar las clases `Reader` y `Dataset`. La primera de ellas, nos permitirá configurar como debemos leer el fichero .csv. La segunda, cargará los datos del fichero a un objeto (`data`).

In [219]:
from surprise import Dataset, Reader
file_path = '/Users/dacilestherbatistagarcia/Documents/Máster/Trabajo Fin de Máster/rating.csv'
reader = Reader(
    line_format='user item rating', sep=',', rating_scale=(1, 5), skip_lines=1
)
# Cargamos los datos del fichero
data = Dataset.load_from_file(file_path, reader=reader)

Una vez que tenemos los datos disponibles, necesitamos prepararlos para nuestro sistema recomendador. Necesitamos dividir los datos en 2 grupos: datos de entrenamiento (`trainset`) y datos de evaluación (`testset`).

Para ello, usaremos la función `train_test_split` que se encuentra dentro del módulo `model_selection`. En este ejemplo vamos a utilizar el 90% de los datos como datos de entrenamiento, y el 10% restante para la evaluación. Esto lo configuramos con el parámetro `test_size` de la función.

In [220]:
# Preparar los datos para evaluar el sistema recomendador
from surprise.model_selection import train_test_split

# Dividimos los datos en 2 datasets, uno para entrenar el sistema, otro para evaluarlo
trainset, testset = train_test_split(data, test_size=0.1)

# Imprimimos el número de items y usuarios que tenemos en cada sitio
print('Usuarios: ', trainset.n_users)
print('Items: ', trainset.n_items)

Usuarios:  6
Items:  34


Con los datos preparados, podemos pasar a entrenar nuestro modelo de recomendación.

La librería surprise nos propone un conjunto de algoritmos basados en factorización de matrices para sistemas recomendadores de filtrado colaborativo basado en modelo.

En este caso usaremos el algoritmo `SVD`. Existen múltiples parámetros que podemos utilizar para configurar el entrenamiento de nuestro algoritmo, pero los más importantes son dos:
- `n_factors`: Número de factores de las matrices p y q
- `n_ecpochs`: Número de iteraciones para entrenar el algoritmo.

Para este ejemplo usaremos 15 factores y 20 iteraciones. Una vez configurado el algoritmo, procedemos a entrenarlo con la función `fit`.

In [221]:
# Configurar el sistema recomendador
from surprise import SVD
# Creamos una instancia del modelo
algorithm = SVD(
    n_factors=15,
    n_epochs=20
)
# Entrenamos el modelo
algorithm.fit(trainset)

Vamos a ver que tal funciona nuestro algoritmo. Para ello vamos a utilizar una de las métricas que nos incluye `surprise`. 

La forma que tenemos de evaluar nuestro sistema recomendador es, hacer una predicción de los ratings de los datos que hemos guardado para la evaluación y, a continuación, aplicar la métrica RMSE.

In [222]:
# Evaluar sistema recomendador
from surprise import accuracy
# Calculamos las predicciones de los ratings de los datos de evaluación
predictions = algorithm.test(testset)
# Obtenemos medidas de error
accuracy.rmse(predictions)

RMSE: 1.3671


1.367085505269531

Vamos a elegir un paciente, para ver que recomendaciones de actividades le haríamos según este algoritmo de sistema recomendador colaborativo

In [223]:
# Fijamos uno de los pacientes que tenemos guardados en el .csv que creamos anteriormente
user_id =str(33)

Vamos a obtener una lista con las actividades recomendadas por este algoritmo a nuestro paciente que ya hemos definido:

In [224]:
from collections import defaultdict
# Predecimos las valoraciones para las actividades que no están valoradas
number_of_predictions = 10
no_rating_set = trainset.build_anti_testset()
pred = algorithm.test(no_rating_set)

# Asignamos las predicciones a cada usuario
top_n = defaultdict(list)
for uid, iid, true_r, est, _ in pred:
    top_n[uid].append((eval(iid),est))
    
# Ordenamos las predicciones de cada usuario, y nos quedamos con las que tienen las predicciones más altas    
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:number_of_predictions]
    
# Imprimimos las predicciones del usuario que definimos anteriormente
for (iid, rating) in top_n[user_id]:
    print("Actividad: {:>40} - Valoración: {}".format((actividades.Nombre_Actividad[iid]), rating))


Actividad:                   Lanzamiento de objetos - Valoración: 2.9374192409062116
Actividad:            Colección de flores y colores - Valoración: 2.929800707306364
Actividad:                           Adivina qué es - Valoración: 2.895876287719911
Actividad:                               La brújula - Valoración: 2.891961977663989
Actividad:               Lectura o escucha dinámica - Valoración: 2.889176824931466
Actividad:               ¿Qué historia te imaginas? - Valoración: 2.8814348576167825
Actividad:                  Desde el lado contrario - Valoración: 2.877561012123259
Actividad:                             Somos espejo - Valoración: 2.865067080525873
Actividad:                           Pongamos orden - Valoración: 2.8573074315253133
Actividad:           Parejas de naipes, boca arriba - Valoración: 2.754876843565099


- Valoraciones y recomendaciones del Sistema Recomendador de Contenido, para el paciente definido

In [225]:
listCon = (gustos_similares_all)
print (listCon)

[(0, 0.5430923677878643), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 0.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 0.35015432264617263), (13, 0.34480647023514094), (14, 1.3387914356463255), (15, 1.3387914356463255), (16, 1.0), (17, 0.5430923677878643), (18, 0.5430923677878643), (19, 0.5430923677878643), (20, 0.5430923677878643), (21, 0.5430923677878643), (22, 0.5430923677878643), (23, 1.0), (24, 1.3387914356463255), (25, 1.6449590536073335), (26, 0.39434244162100385), (27, 1.0), (28, 0.5430923677878643), (29, 0.5430923677878643), (30, 1.3607165830865466), (31, 1.3607165830865466), (32, 0.5981561475248088), (33, 1.3387914356463255), (34, 0.8988446437908375), (35, 0.5795103999013562), (36, 0.5795103999013562), (37, 0.5795103999013562), (38, 1.0), (39, 0.5795103999013562), (40, 0.5795103999013562), (41, 0.467493699170923), (42, 0.467493699170923), (43, 1.0), (44, 1.0), (45, 0.467493699170923), (46, 0.467493699170923), (47, 1.3607165830865466), (48, 0.5795103999

- Valoraciones y recomendaciones del Sistema Recomendador Colaborativo, para el paciente definido

In [226]:
listCol = (top_n[user_id])
print (listCol)

[(31, 2.9374192409062116), (61, 2.929800707306364), (18, 2.895876287719911), (36, 2.891961977663989), (58, 2.889176824931466), (55, 2.8814348576167825), (45, 2.877561012123259), (34, 2.865067080525873), (62, 2.8573074315253133), (15, 2.754876843565099)]


# Sistema Recomendador Híbrido

Para esto vamos a combinar los dos sistemas recomendadores que hemos creado anteriormente. Cada sistema tendrá un peso dentro de este sistema Híbrido, el cual vamos a definirlo a continuación:

In [227]:
def obtener_recomendacion (user):
    pesoContenido = 0.4
    pesoColaborativo = 0.6
    listaCon = (gustos_similares_all)
    listaCol = (top_n[user])
    rangoCon = len(listaCon)
    rangoCol = len(listaCol)
    items_peso = {}
    
    for i in range(rangoCol):
        try:
            nuevoCol = listaCol[i][1] * pesoColaborativo
            items_peso[listaCol[i][0]] = max(items_peso[listaCol[i][0]],nuevoCol)
        except KeyError:
            items_peso[listaCol[i][0]] = listaCol[i][1] * pesoColaborativo
            
    for i in range(rangoCon):
        try:
            nuevoCon = listaCon[i][1] * pesoContenido
            items_peso[listaCon[i][0]] = max(items_peso[listaCon[i][0]],nuevoCon)
        except KeyError:
            items_peso[listaCon[i][0]] = listaCon[i][1] * pesoContenido
    
    act_recomendadas = sorted(items_peso.items(), key=lambda x: x[1], reverse = True)
    
    return act_recomendadas

recomendacion = obtener_recomendacion (user_id)
print(recomendacion)
    

[(31, 1.762451544543727), (61, 1.7578804243838182), (18, 1.7375257726319464), (36, 1.7351771865983934), (58, 1.7335060949588796), (55, 1.7288609145700695), (45, 1.7265366072739552), (34, 1.7190402483155236), (62, 1.714384458915188), (15, 1.6529261061390594), (25, 0.6579836214429334), (30, 0.5442866332346187), (47, 0.5442866332346187), (14, 0.5355165742585303), (24, 0.5355165742585303), (33, 0.5355165742585303), (1, 0.4), (2, 0.4), (3, 0.4), (4, 0.4), (5, 0.4), (7, 0.4), (8, 0.4), (9, 0.4), (10, 0.4), (11, 0.4), (16, 0.4), (23, 0.4), (27, 0.4), (38, 0.4), (43, 0.4), (44, 0.4), (53, 0.4), (32, 0.23926245900992354), (35, 0.2318041599605425), (37, 0.2318041599605425), (39, 0.2318041599605425), (40, 0.2318041599605425), (48, 0.2318041599605425), (49, 0.2318041599605425), (50, 0.2318041599605425), (51, 0.2318041599605425), (54, 0.2318041599605425), (0, 0.21723694711514574), (17, 0.21723694711514574), (19, 0.21723694711514574), (20, 0.21723694711514574), (21, 0.21723694711514574), (22, 0.2172

### Imprimimos por pantalla el resultado de este Sistema Recomendador Híbrido. Para ello, vamos a imprimir las 10 recomendaciones mejor valoradas. 

In [228]:
# Vamos a imprimir por pantalla las 10 recomendaciones con mayor puntuación
recomendacion = recomendacion [:10]
for (iid, rating) in recomendacion:
    print("Actividad: {:>40} - Valoración: {}".format((actividades.Nombre_Actividad[iid]), rating))

Actividad:                   Lanzamiento de objetos - Valoración: 1.762451544543727
Actividad:            Colección de flores y colores - Valoración: 1.7578804243838182
Actividad:                           Adivina qué es - Valoración: 1.7375257726319464
Actividad:                               La brújula - Valoración: 1.7351771865983934
Actividad:               Lectura o escucha dinámica - Valoración: 1.7335060949588796
Actividad:               ¿Qué historia te imaginas? - Valoración: 1.7288609145700695
Actividad:                  Desde el lado contrario - Valoración: 1.7265366072739552
Actividad:                             Somos espejo - Valoración: 1.7190402483155236
Actividad:                           Pongamos orden - Valoración: 1.714384458915188
Actividad:           Parejas de naipes, boca arriba - Valoración: 1.6529261061390594
